In [1]:
import json
import spacy
from nlp_utils import *
import numpy as np

import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader



In [2]:
with open("data.json", "r") as f:
    data = json.load(f)["intents"]
    

In [3]:
len(data)

22

- dictionary 


In [4]:
data_set = []

nlp = spacy.load("en_core_web_lg")

for intent in data:
    tag = intent["intent"]
    for pattern in intent["text"]:
        w = tokenize_to_vec(pattern, nlp)
        
        data_set.append((w, tag))



In [5]:
tag_translator = {}
count = 0

for (vec, tag) in data_set:
    if tag not in tag_translator.keys():
        tag_translator[tag] = count
        count += 1 

In [6]:
tag_translator

{'Greeting': 0,
 'GreetingResponse': 1,
 'CourtesyGreeting': 2,
 'CourtesyGreetingResponse': 3,
 'CurrentHumanQuery': 4,
 'NameQuery': 5,
 'RealNameQuery': 6,
 'TimeQuery': 7,
 'Thanks': 8,
 'NotTalking2U': 9,
 'UnderstandQuery': 10,
 'Shutup': 11,
 'Swearing': 12,
 'GoodBye': 13,
 'CourtesyGoodBye': 14,
 'WhoAmI': 15,
 'Clever': 16,
 'Gossip': 17,
 'Jokes': 18,
 'PodBayDoor': 19,
 'PodBayDoorResponse': 20,
 'SelfAware': 21}

In [7]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(data_set)
        self.X_data = []
        self.y_data = []
        
        for (words, tag) in data_set:
            self.X_data.append(words)
            self.y_data.append(tag_translator[tag])
        self.X_data = np.array(self.X_data)
        self.y_data = np.array(self.y_data)
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
        

In [12]:
# hyperparameters
batch_size = 10
input_size = 300
hidden_size = 100
num_classes = 22
learning_rate = 0.001
num_epochs = 500

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)


device = torch.device("cuda")




model = NeuralNet(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) %10 == 0:
        print(f"epoch {epoch+1}/{num_epochs}, loss={loss.item():.8f}")
        
              
print(f"final loss={loss.item():.8f}")      

epoch 10/500, loss=0.13082622
epoch 20/500, loss=0.12956068
epoch 30/500, loss=0.00431572
epoch 40/500, loss=0.00629344
epoch 50/500, loss=0.00214917
epoch 60/500, loss=0.00173272
epoch 70/500, loss=0.00181919
epoch 80/500, loss=0.00122955
epoch 90/500, loss=0.00034631
epoch 100/500, loss=0.00011518
epoch 110/500, loss=0.00067998
epoch 120/500, loss=0.00022627
epoch 130/500, loss=0.00005110
epoch 140/500, loss=0.00055361
epoch 150/500, loss=0.00060643
epoch 160/500, loss=0.00003147
epoch 170/500, loss=0.00001327
epoch 180/500, loss=0.00025657
epoch 190/500, loss=0.00002634
epoch 200/500, loss=0.00013477
epoch 210/500, loss=0.00153172
epoch 220/500, loss=0.00126074
epoch 230/500, loss=0.00002412
epoch 240/500, loss=0.00000584
epoch 250/500, loss=0.00013672
epoch 260/500, loss=0.00003672
epoch 270/500, loss=0.00001792
epoch 280/500, loss=0.00002066
epoch 290/500, loss=0.00013823
epoch 300/500, loss=0.00000719
epoch 310/500, loss=0.00002519
epoch 320/500, loss=0.00010255
epoch 330/500, lo

In [13]:
save_data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "num_classes": num_classes,
    "tag_translator": tag_translator
}

torch.save(save_data, "model.save")

